In [ ]:
# Imports
import os
import json
import numpy as np
import networkx as nx
from natsort import natsorted
import coptpy as cp
from coptpy import COPT

In [ ]:
def read_nxgraph(filename: str) -> nx.Graph(): # type: ignore
    graph = nx.Graph()
    with open(filename, 'r') as file:
        line = file.readline()
        is_first_line = True
        while line is not None and line != '':
            if '//' not in line:
                if is_first_line:
                    strings = line.split(" ")
                    num_nodes = int(strings[0])
                    num_edges = int(strings[1])
                    nodes = list(range(num_nodes))
                    graph.add_nodes_from(nodes)
                    is_first_line = False
                else:
                    node1, node2, weight = line.split()
                    graph.add_edge(int(node1), int(node2), weight=weight)
            line = file.readline()
    return graph

In [ ]:
def copt_maxcut(graph):

    # Create QUBO matrix
    nodes = len(list(graph.nodes))
    J = nx.to_numpy_array(graph)

    # Create COPT environment
    env = cp.Envr()
    
    # Construct model
    model = env.createModel("qubo")

    # Set time limit 1 hr
    model.setLogFile("../logs/copt.log")
    model.setParam(COPT.Param.TimeLimit, 3600.0)
    model.setParam(COPT.Param.RelGap, 0.0)

    # Create variable for each vertex
    x = [ model.addVar(vtype=COPT.BINARY, name=f"x_{i}") for i in range(nodes) ]
                
    # Linearize x_i * x_j to y_ij
    y = np.zeros((nodes, nodes), dtype=object)
    for i in range(nodes):
        for j in range(i +1, nodes):
            y[i, j] = model.addVar(vtype=COPT.BINARY, name=f"y_{i},{j}")
            model.addConstr(y[i, j] <= x[i])
            model.addConstr(y[i, j] <= x[j])
            model.addConstr(y[i, j] >= x[i] + x[j] - 1)

    # Objective function
    model.setObjective(cp.quicksum(
        -J[i, j] * ( (4 * y[i,j]) + (-2 * x[i]) + (-2 * x[j]) + 1 )
        for i in range(nodes) for j in range(i + 1, nodes) if J[i, j] != 0.0 
    ), sense=COPT.MINIMIZE)

    # Solve
    model.solve()
    sol = ""
    for i in range(nodes):
        sol += f"{int(x[i].x)}"
    
    return model.BestObj, sol

In [ ]:
# graph = read_nxgraph("../data/RL/spin_glass/10x10/GSRL_100_ID2.txt")
# obj_val, sol = copt_maxcut(graph)

In [ ]:
# Parse all files in data_dir 
data_dir = f"../data/vna/EA/EA_40x40"
input_files = [ f for f in os.listdir(data_dir) ]
input_files = natsorted(input_files)

# Write directories & files
results_dir = f"../results/{'/'.join(data_dir.split('/')[2:])}"
solutions_dir = f"../solutions/{'/'.join(data_dir.split('/')[2:])}"
os.makedirs(results_dir, exist_ok=True)
os.makedirs(solutions_dir, exist_ok=True)

i = 0
for file in input_files:
    i += 1
    if i > 25: break

    graph = read_nxgraph(f'{data_dir}/{file}') 
    obj_val, sol = copt_maxcut(graph)

    with open(f"{results_dir}/COPT.txt", "a") as f:
        f.write(f"{obj_val}\n")

    with open(f"{solutions_dir}/COPT.txt", "a") as f:
        f.write(f"{sol}\n")
